In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# Load Data

In [2]:
def load_scenario_data(h5_file_path, scenario_num):
    key = f"scenario_{scenario_num}/data"
    with pd.HDFStore(h5_file_path, mode='r') as store:
        if key not in store:
            return None  # Scenario not found
        df = store[key]
        metadata = store.get_storer(key).attrs.metadata
    return {"dataset": df, "metadata": metadata}

In [3]:
store_files = [
    "synthetic_data/RCT_0_5.h5",
    "synthetic_data/RCT_0_05.h5",
    "synthetic_data/e_X.h5",
    "synthetic_data/e_X_U.h5",
    "synthetic_data/e_X_info_censor.h5",
    "synthetic_data/e_X_U_info_censor.h5"
]

experiment_setups = {}

for path in store_files:
    base_name = os.path.splitext(os.path.basename(path))[0]  # e.g. RCT_0_5
    scenario_dict = {}
    for scenario in range(1, 11):
        try:
            result = load_scenario_data(path, scenario)
            if result is not None:
                scenario_dict[f"scenario_{scenario}"] = result
        except Exception as e:
            # Log or ignore as needed
            continue
    experiment_setups[base_name] = scenario_dict

In [4]:
experiment_setups['RCT_0_5']['scenario_1']['dataset'].head()

,id,observed_time,event,W,X1,X2,X3,X4,X5,U1,U2,T0,T1,T,C
0,0,0.054267,1,0,0.135488,0.887852,0.932606,0.445568,0.388236,0.151609,0.205535,0.054267,0.061394,0.054267,1.803019
1,1,0.732630,1,1,0.257596,0.657368,0.492617,0.964238,0.800984,0.597208,0.255785,0.228566,0.732630,0.732630,1.689546
2,2,0.162856,1,1,0.455205,0.801058,0.041718,0.769458,0.003171,0.370382,0.223214,0.176016,0.162856,0.162856,1.256329
3,3,0.050340,1,1,0.292809,0.610914,0.913027,0.300115,0.248599,0.038464,0.409829,0.381909,0.050340,0.050340,1.241777
4,4,0.524607,1,0,0.666392,0.987533,0.468270,0.123287,0.916031,0.342961,0.791330,0.524607,1.121968,0.524607,1.516613


In [5]:
experiment_repeat_setups = pd.read_csv("synthetic_data/idx_split.csv").set_index("idx")
experiment_repeat_setups

,random_idx0,random_idx1,random_idx2,random_idx3,random_idx4,random_idx5,random_idx6,random_idx7,random_idx8,random_idx9
idx,,,,,,,,,,
0,47390,5618,14210,46970,4203,16369,24535,45204,45725,45885
1,38566,46218,39045,7253,22759,34401,28889,38471,45822,37471
2,32814,20226,40012,4854,27351,39165,25359,14516,25717,29860
3,41393,39492,27153,19041,33009,19822,21243,41228,955,23901
4,12564,17823,48976,18458,22756,28169,45851,36620,29824,12711
...,...,...,...,...,...,...,...,...,...,...
49995,15948,39245,30779,48178,45056,4892,528,7486,31042,38267
49996,11102,29624,40779,3136,45904,41903,45682,36621,33204,38070
49997,16338,8986,19293,35651,10172,17947,38843,18310,2765,12581


# Run Experiments

In [6]:
from models_causal_impute.meta_learners import TLearner, SLearner, XLearner
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [7]:
def prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list):
    split_results = {}

    for col in random_idx_col_list:
        random_idx = experiment_repeat_setups[col].values
        test_ids = random_idx[-5000:]
        train_ids = random_idx[:-5000]

        X_cols = [c for c in dataset_df.columns if c.startswith("X") and c[1:].isdigit()]
        
        train_df = dataset_df[dataset_df['id'].isin(train_ids)]
        test_df = dataset_df[dataset_df['id'].isin(test_ids)]

        X_train = train_df[X_cols].to_numpy()
        W_train = train_df["W"].to_numpy()
        Y_train = train_df[["observed_time", "event"]].to_numpy()

        X_test = test_df[X_cols].to_numpy()
        W_test = test_df["W"].to_numpy()
        Y_test = test_df[["observed_time", "event"]].to_numpy()

        CATE_test_true = (test_df["T1"] - test_df["T0"]).to_numpy()

        split_results[col] = (X_train, W_train, Y_train, X_test, W_test, Y_test, CATE_test_true)

    return split_results

In [8]:
random_idx_col_list = ["random_idx0"]
failure_times_grid_size = 500
output_pickle_path = "causal_impute_t_learner_random_idx0_train_45000.pkl"

base_regressors = ['ridge', 'lasso', 'rf', 'gbr', 'xgb']

for setup_name, setup_dict in tqdm(experiment_setups.items(), desc="Experiment Setups"):
    for scenario_key in tqdm(setup_dict, desc=f"{setup_name} Scenarios"):
        dataset_df = setup_dict[scenario_key]["dataset"]
        split_dict = prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list)

        if "result" not in experiment_setups[setup_name][scenario_key]:
            experiment_setups[setup_name][scenario_key]["result"] = {}


        # For each base model, we will run the TLearner
        for base_model in tqdm(base_regressors, desc="Base Models", leave=False):
            print(f"Running {base_model} for {setup_name} - {scenario_key}")
            
            # Store placeholder for later population
            experiment_setups[setup_name][scenario_key]["result"][base_model] = {}

            
            start_time = time.time()

            for col in random_idx_col_list:
                X_train, W_train, Y_train, X_test, W_test, Y_test, CATE_test_true = split_dict[col]

                ##################################
                # TODO: Imputation Step
                Y_train = Y_train[:, 0]  # observed_time
                Y_test  = Y_test[:, 0]   # observed_time
                ##################################
                
                
                learner = TLearner(base_model_name=base_model)
                learner.fit(X_train, W_train, Y_train)
                mse_test, CATE_test_pred = learner.evaluate(X_test, CATE_test_true)

                experiment_setups[setup_name][scenario_key]["result"][base_model][col] = {
                    "CATE_true": CATE_test_true,
                    "CATE_Pred": CATE_test_pred,
                    "CATE_MSE": mse_test
                }

            end_time = time.time()

            experiment_setups[setup_name][scenario_key]["result"][base_model]["average"] = {
                "mean_CATE_MSE": np.mean([experiment_setups[setup_name][scenario_key]["result"][base_model][col]["CATE_MSE"]
                                        for col in random_idx_col_list]),
                "std_CATE_MSE":  np.std( [experiment_setups[setup_name][scenario_key]["result"][base_model][col]["CATE_MSE"]
                                        for col in random_idx_col_list]),
                "runtime": (end_time - start_time) / len(random_idx_col_list)
            }

            # Save progress to disk
            # with open(output_pickle_path, "wb") as f:
                # pickle.dump(experiment_setups, f)
            
            # break
        # break
    # break
            


Experiment Setups:   0%|          | 0/6 [00:00<?, ?it/s]


Running ridge for RCT_0_5 - scenario_1


Running lasso for RCT_0_5 - scenario_1


Running rf for RCT_0_5 - scenario_1


Running gbr for RCT_0_5 - scenario_1


...

Running ridge for e_X_U_info_censor - scenario_9
Running lasso for e_X_U_info_censor - scenario_9


Running rf for e_X_U_info_censor - scenario_9


Running gbr for e_X_U_info_censor - scenario_9


Running xgb for e_X_U_info_censor - scenario_9




Experiment Setups: 100%|██████████| 6/6 [33:27<00:00, 334.58s/it]


In [11]:
def summarize_experiment_results(experiment_setups, base_regressors):
    records = []

    for setup_name, setup_dict in experiment_setups.items():
        for scenario_key in setup_dict:
            row = {
                ("setup_name", ""): setup_name,
                ("scenario_key", ""): scenario_key
            }

            for base_model in base_regressors:
                avg_result = setup_dict[scenario_key].get("result", {}).get(base_model, {}).get("average", {})
                mean_mse = avg_result.get("mean_CATE_MSE", np.nan)
                std_mse = avg_result.get("std_CATE_MSE", np.nan)
                runtime = avg_result.get("runtime", np.nan)

                row[(base_model, "mean_CATE_mse")] = round(mean_mse, 4) if not pd.isna(mean_mse) else np.nan
                # row[(base_model, "std_CATE_mse")] = round(std_mse, 4) if not pd.isna(std_mse) else np.nan
                row[(base_model, "runtime [s]")] = round(runtime, 4) if not pd.isna(runtime) else np.nan

            records.append(row)

    df = pd.DataFrame.from_records(records)
    df.columns = pd.MultiIndex.from_tuples(df.columns)
    return df


In [12]:
print("T-Learner Results")
summary_df = summarize_experiment_results(experiment_setups, base_regressors)
summary_df

T-Learner Results


setup_name scenario_key         ridge                     lasso  \
                                   mean_CATE_mse runtime [s] mean_CATE_mse   
0             RCT_0_5   scenario_1        0.6923      1.4245        0.6924   
1             RCT_0_5   scenario_2       15.8143      0.2101       15.8140   
2             RCT_0_5   scenario_3       14.5100      0.2655       14.5073   
3             RCT_0_5   scenario_5       14.6204      0.1831       14.6207   
4             RCT_0_5   scenario_6       15.0242      0.1852       15.0299   
5             RCT_0_5   scenario_8       17.2597      0.1934       17.2528   
6             RCT_0_5   scenario_9       43.1339      0.1982       43.1336   
7             RCT_0_5  scenario_10       15.8455      0.1883       15.8559   
8            RCT_0_05   scenario_1        0.6919      0.2706        0.6918   
9            RCT_0_05   scenario_2       15.8345      0.2784       15.8404   
10           RCT_0_05   scenario_3       14.5072      0.2593       14.5032   
11           RCT_0_05   scenario_5       14.6307      0.2521       14.6307   
12           RCT_0_05   scenario_6       15.0230      0.2600       15.0275   
13           RCT_0_05   scenario_8       17.2416      0.3001       17.2538   
14           RCT_0_05   scenario_9       43.1115      0.2502       43.1101   
15           RCT_0_05  scenario_10       15.8764      0.2299       15.8826   
16                e_X   scenario_1        0.6932      0.1997        0.6934   
17                e_X   scenario_2       15.8061      0.1927       15.8079   
18                e_X   scenario_3       14.5287      0.1963       14.5223   
19                e_X   scenario_5       14.6475      0.2037       14.6448   
20                e_X   scenario_6       15.0196      0.2183       15.0205   
21                e_X   scenario_8       17.2591      0.1803       17.3012   
22                e_X   scenario_9       43.1227      0.1711       43.1245   
23                e_X  scenario_10       15.8670      0.1706       15.8664   
24              e_X_U   scenario_1        0.6928      0.2006        0.6929   
25              e_X_U   scenario_2       15.8173      0.1777       15.8191   
26              e_X_U   scenario_3       14.5188      0.1888       14.5204   
27              e_X_U   scenario_5       14.6957      0.1871       14.6956   
28              e_X_U   scenario_6       15.0279      0.1915       15.0324   
29              e_X_U   scenario_8       17.2711      0.1814       17.2804   
30              e_X_U   scenario_9       43.1229      0.1822       43.1235   
31              e_X_U  scenario_10       15.8662      0.1772       15.8637   
32    e_X_info_censor   scenario_1        0.6864      0.1924        0.6865   
33    e_X_info_censor   scenario_2       15.7771      0.1932       15.7785   
34    e_X_info_censor   scenario_3       15.5078      0.2091       15.4881   
35    e_X_info_censor   scenario_8       17.4853      0.1706       17.4666   
36    e_X_info_censor   scenario_9       42.4570      0.2068       42.4574   
37  e_X_U_info_censor   scenario_1        0.6860      0.1860        0.6861   
38  e_X_U_info_censor   scenario_2       15.7921      0.1724       15.7933   
39  e_X_U_info_censor   scenario_3       15.4206      0.1688       15.4119   
40  e_X_U_info_censor   scenario_8       17.5402      0.1706       17.5400   
41  e_X_U_info_censor   scenario_9       42.4331      0.1713       42.4324   

                          rf                       gbr              \
   runtime [s] mean_CATE_mse runtime [s] mean_CATE_mse runtime [s]   
0       0.3251        0.6900     24.9766        0.6906     19.5486   
1       0.2526       15.8130     26.2517       15.8108     19.4713   
2       0.4615       14.5023     25.2898       14.5142     19.5803   
3       0.2401       14.6803     23.1341       14.6664     20.8295   
4       0.2494       15.0758     24.9503       15.0699     19.5163   
5       0.2498       17.2517     31.8656       17.2482     20.6778   
6       0.2858    